In [1]:
import pandas as pd
import numpy as np
import microdf as mdf
import plotly.express as px
import statsmodels.api as sm
from ubicenter import format_fig

In [2]:
BUDGET = 1e9  # $1 billion.
REACH = 500e3  # 500,000 adult New Yorkers.
RUN_SIMS = False  # To re-run (takes ~5min).

# Read data per person per year.
py = pd.read_csv("data/nys_cps.csv.gz")
py.columns = py.columns.str.lower()
py.inctot.replace({999999999: 0}, inplace=True)
py.adjginc.replace({99999999: 0}, inplace=True)
# 2014 was double-sampled.
py.loc[py.year == 2014, ["asecwt", "spmwt"]] /= 2
# ASEC shows survey year.
py.year -= 1

LABELS = {
    "poverty_rate": "Poverty rate",
    "deep_poverty_rate": "Deep poverty rate",
    "adult_deep_poverty_rate": "Deep adult poverty rate",
    "child_deep_poverty_rate": "Deep child poverty rate",
    "poverty_gap": "Poverty gap",
    "squared_poverty_gap": "Poverty severity",
    "deep_poverty_gap": "Deep poverty gap",
    "pov_ratio_guarantee": "Maximum benefit as a share of SPM poverty threshold",
    "phase_out_rate": "Benefit phase-out rate with respect to SPM resources",
    "phase_out_rate_pct": "Benefit phase-out rate with respect to SPM resources",
    "constraint": "Constraint",
    "cost_b": "Cost ($B)",
    "adult_reach": "Adult recipients",
    "pct_diff": "Policy impact",
    "yearly": "Average yearly change, 2009-2019",
    "name": "Metric",
    "years_progress": "Policy impact / average yearly change",
}

LIGHTER_BLUE = "#ABCEEB"  # Blue 100.
LIGHT_BLUE = "#49A6E2"  # Blue 500.
BLUE = "#1976D2"  # Blue 700.
DARK_BLUE = "#0F4AA1"  # Blue 900.
GRAY = "#BDBDBD"

COLOR_MAP = {
    "Poverty rate": LIGHT_BLUE,
    "Deep poverty rate": DARK_BLUE,
    "Poverty gap": LIGHT_BLUE,
    "Deep poverty gap": DARK_BLUE,
    "25%": LIGHTER_BLUE,
    "50%": LIGHT_BLUE,
    "100%": DARK_BLUE,
    "$1 billion cost": GRAY,
    "500,000 adult recipients": BLUE,
}

In [3]:
NYC_COUNTY = pd.DataFrame(columns=["fips", "county", "pop_m"])
NYS_FIPS = 36
# Per https://en.wikipedia.org/wiki/List_of_counties_in_New_York
NYC_COUNTY.loc["Manhattan"] = pd.Series({"fips": 61, "county": "New York", "pop_m": 1.632})
NYC_COUNTY.loc["Brooklyn"] = pd.Series({"fips": 47, "county": "Kings", "pop_m": 2.59})
NYC_COUNTY.loc["The Bronx"] = pd.Series({"fips": 5, "county": "Bronx", "pop_m": 1.435})
NYC_COUNTY.loc["Staten Island"] = pd.Series({"fips": 85, "county": "Richmond", "pop_m": 0.475})
NYC_COUNTY.loc["Queens"] = pd.Series({"fips": 81, "county": "Queens", "pop_m": 2.287})
NYC_COUNTY["full_fips"] = NYS_FIPS * 1000 + NYC_COUNTY.fips

py = py[py.county.isin(NYC_COUNTY.full_fips)][
    ["county", "asecwt", 'age', 'sex', 'race', 'hispan', 'inctot', 'spmwt', 'spmtotres', 'spmthresh',
       'spmfamunit', 'adjginc', 'ftotval', 'spmftotval', 'year', 'spmgeoadj']]
py["adult"] = py.age > 17
py["child"] = py.age <= 17
py["spmratio"] = py.spmtotres / py.spmthresh
py["poor"] = py.spmratio < 1
py["deep_poor"] = py.spmratio < 0.5
# py[]
NYC_COUNTY["asec_pop_m"] = NYC_COUNTY.full_fips.apply(
    lambda x: py[(py.county == x) & (py.year == py.year.max())].asecwt.sum() / 1e6
)
NYC_COUNTY

,fips,county,pop_m,full_fips,asec_pop_m
Manhattan,61,New York,1.632,36061,1.634743
Brooklyn,47,Kings,2.590,36047,2.634859
The Bronx,5,Bronx,1.435,36005,1.586280
Staten Island,85,Richmond,0.475,36085,0.497162
Queens,81,Queens,2.287,36081,1.869408


In [4]:
def year_stats(p):
    s = p[["spmfamunit", "year", "spmtotres", "spmthresh", "spmwt"]].drop_duplicates()
    s["spmtotres_pos"] = np.maximum(s.spmtotres, 0)
    return pd.Series({
        "spmgeoadj": mdf.weighted_mean(p, "spmgeoadj", "asecwt"),
        "poverty_rate": mdf.weighted_mean(p, "poor", "asecwt"),
        "deep_poverty_rate": mdf.weighted_mean(p, "deep_poor", "asecwt"),
        "poor_adults": mdf.weighted_sum(p[p.adult], "poor", "asecwt"),
        "deep_poor_adults": mdf.weighted_sum(p[p.adult], "deep_poor", "asecwt"),
        "child_poverty_rate": mdf.weighted_mean(p[p.child], "poor", "asecwt"),
        "child_deep_poverty_rate": mdf.weighted_mean(p[p.child], "deep_poor", "asecwt"),
        "adult_poverty_rate": mdf.weighted_mean(p[p.adult], "poor", "asecwt"),
        "adult_deep_poverty_rate": mdf.weighted_mean(p[p.adult], "deep_poor", "asecwt"),
        "poverty_gap": mdf.poverty_gap(s, "spmtotres", "spmthresh", "spmwt"),
        "squared_poverty_gap": mdf.squared_poverty_gap(s, "spmtotres", "spmthresh", "spmwt"),
        "deep_poverty_gap": mdf.deep_poverty_gap(s, "spmtotres", "spmthresh", "spmwt"),
        "gini": mdf.gini(s, "spmtotres", "spmwt"),
        "population": p.asecwt.sum(),
        "spm_units": s.spmwt.sum(),
        "survey_population": p.shape[0],
        "survey_spm_units": s.shape[0],
    })
year = py.groupby("year").apply(year_stats)
year["year"] = year.index


In [35]:
pov_rate_trend = year.melt("year", ["poverty_rate", "deep_poverty_rate"])
pov_rate_trend.variable = pov_rate_trend.variable.map(LABELS)
fig = px.line(
    pov_rate_trend,
    "year",
    "value",
    color="variable",
    color_discrete_map=COLOR_MAP,
    labels=LABELS,
    title="Poverty rate trends in NYC (SPM)"
)
fig.update_layout(
    yaxis_tickformat=".1%",
    xaxis_title="",
    yaxis_title="",
    legend_title="",
    yaxis_range=[0, 0.3]
)
format_fig(fig)

In [7]:
pov_gap_trend = year.melt("year", ["poverty_gap", "deep_poverty_gap"])
pov_gap_trend.variable = pov_gap_trend.variable.map(LABELS)
fig = px.line(
    pov_gap_trend,
    "year",
    "value",
    color="variable",
    color_discrete_map=COLOR_MAP,
    labels=LABELS,
    title="Poverty gap trends in NYC (SPM)")
fig.update_layout(yaxis_tickprefix="$", xaxis_title="", yaxis_title="", legend_title="",
    yaxis_range=[0, 1e10])
format_fig(fig)

In [8]:
def pct_chg_trend(col):
    # return (year[col].iloc[-1] / year[col].iloc[0]) ** (1 / (year.shape[0] - 1)) - 1
    return np.mean((year[col] / year[col].shift(1))) - 1

pct_chg_trend("deep_poverty_gap")

-0.032960951141618566

In [9]:
def trend(col):
    if col in year.columns:
        # Coefficient on year of a regression of the difference in a variable.
        return sm.OLS(
            year[col] / year[col].iloc[0],
            sm.add_constant(year.index)
        ).fit().params["x1"]
    return None

In [16]:
# Use three years of data to smooth out given small sample size.
p = py[py.year.isin([2017, 2018, 2019])].copy()
p[["asecwt", "spmwt"]] /= 3

P_S_MERGE_COLS = ["year", "spmfamunit"]  # SPM unit keys from person.
SPMU_COLS = ["spmtotres", "spmthresh", "spmratio", "spmwt", "spmftotval"]
SPMU_AGGS = ["adult", "child"]

s = p.groupby(P_S_MERGE_COLS + SPMU_COLS)[SPMU_AGGS].sum().reset_index()

def phase_out(amount, rate, respect_to):
    return np.maximum(0, amount - respect_to * rate)



def policy(pov_ratio_guarantee, phase_out_rate):
    """ Computes features of a negative income tax.

    Args:
        pov_ratio_guarantee: Maximum benefit as a share of a SPM unit's
            poverty threshold.
        phase_out_rate: Phase-out rate with respect to a SPM unit's resources.
    """
    s["max_transfer"] = s.spmthresh * pov_ratio_guarantee
    s["transfer"] = phase_out(s.max_transfer, phase_out_rate,
                              np.maximum(s.spmtotres, 0))
    s["spmtotres_r"] = s.spmtotres + s.transfer
    s["spmtotres_r_pos"] = np.maximum(s.spmtotres_r, 0)
    p2 = p.merge(
        s[P_S_MERGE_COLS + ["transfer", "spmtotres_r", "spmtotres_r_pos"]],
        on=P_S_MERGE_COLS
    )
    return pd.Series({
        # Variables for defining policies.
        "cost_b": mdf.weighted_sum(s, "transfer", "spmwt") / 1e9,
        "adult_reach": p2[(p2.transfer > 0) & p2.adult].asecwt.sum(),
        # Variables for evaluation.
        "spmu_reach": s[s.transfer > 0].spmwt.sum(),
        "child_reach": p2[(p2.transfer > 0) & p2.child].asecwt.sum(),
        "gini": mdf.gini(s, "spmtotres_r", "spmwt"),
        "poverty_rate": mdf.poverty_rate(p2, "spmtotres_r", "spmthresh", "asecwt"),
        "deep_poverty_rate": mdf.deep_poverty_rate(p2, "spmtotres_r", "spmthresh", "asecwt"),
        "child_poverty_rate": mdf.poverty_rate(p2[p2.child], "spmtotres_r", "spmthresh", "asecwt"),
        "child_deep_poverty_rate": mdf.deep_poverty_rate(p2[p2.child], "spmtotres_r", "spmthresh", "asecwt"),
        "adult_poverty_rate": mdf.poverty_rate(p2[p2.adult], "spmtotres_r", "spmthresh", "asecwt"),
        "adult_deep_poverty_rate": mdf.deep_poverty_rate(p2[p2.adult], "spmtotres_r", "spmthresh", "asecwt"),
        "poverty_gap": mdf.poverty_gap(s, "spmtotres_r_pos", "spmthresh", "spmwt"),
        "squared_poverty_gap": mdf.squared_poverty_gap(s, "spmtotres_r_pos", "spmthresh", "spmwt"),
        "deep_poverty_gap": mdf.deep_poverty_gap(s, "spmtotres_r_pos", "spmthresh", "spmwt"),
    })

if RUN_SIMS:
    policies = mdf.cartesian_product({
        "pov_ratio_guarantee": np.arange(0.01, 0.51, 0.01),
        "phase_out_rate": np.arange(0, 1.01, 0.01)
        })
    policies = pd.concat([policies,
        policies.apply(lambda x: policy(x.pov_ratio_guarantee, x.phase_out_rate), axis=1)],
        axis=1)
    # Takes ~5min to run.
    policies.to_csv("data/policies.csv", index=False)
else:
    policies = pd.read_csv("data/policies.csv")
policies["phase_out_rate_pct"] = (policies.phase_out_rate * 100).astype(int).astype(str) + "%"
policies

,pov_ratio_guarantee,phase_out_rate,cost_b,adult_reach,spmu_reach,child_reach,gini,poverty_rate,deep_poverty_rate,child_poverty_rate,child_deep_poverty_rate,adult_poverty_rate,adult_deep_poverty_rate,poverty_gap,squared_poverty_gap,deep_poverty_gap,phase_out_rate_pct
0,0.01,0.00,0.860683,6.660054e+06,3.712855e+06,1.739910e+06,0.487076,0.197996,0.047976,0.230656,0.038919,0.189464,0.050342,6.372123e+09,9.435255e+13,1.482777e+09,0%
1,0.01,0.01,0.064600,1.285708e+06,7.512678e+05,4.091063e+05,0.488207,0.201764,0.049370,0.235131,0.040396,0.193048,0.051714,6.479488e+09,9.568039e+13,1.491369e+09,1%
2,0.01,0.02,0.028903,3.472183e+05,2.329493e+05,7.028474e+04,0.488380,0.201764,0.049703,0.235131,0.040396,0.193048,0.052134,6.515185e+09,9.643910e+13,1.500395e+09,2%
3,0.01,0.03,0.023849,2.231429e+05,1.684367e+05,4.841986e+04,0.488410,0.201764,0.049703,0.235131,0.040396,0.193048,0.052134,6.520239e+09,9.662273e+13,1.505449e+09,3%
4,0.01,0.04,0.021741,1.861716e+05,1.452380e+05,3.233471e+04,0.488422,0.201764,0.049703,0.235131,0.040396,0.193048,0.052134,6.522347e+09,9.670770e+13,1.507557e+09,4%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5045,0.50,0.96,1.484681,3.713384e+05,2.469092e+05,7.423493e+04,0.479748,0.201764,0.008381,0.235131,0.002450,0.193048,0.009930,5.059407e+09,5.152153e+13,8.412901e+07,96%
5046,0.50,0.97,1.474007,3.700070e+05,2.462435e+05,7.354813e+04,0.479805,0.201764,0.008381,0.235131,0.002450,0.193048,0.009930,5.070081e+09,5.180054e+13,8.412901e+07,97%
5047,0.50,0.98,1.463818,3.563081e+05,2.376609e+05,7.233507e+04,0.479859,0.201764,0.008381,0.235131,0.002450,0.193048,0.009930,5.080270e+09,5.207123e+13,8.412901e+07,98%
5048,0.50,0.99,1.454281,3.489403e+05,2.346713e+05,7.028474e+04,0.479910,0.201764,0.008381,0.235131,0.002450,0.193048,0.009930,5.089807e+09,5.232758e+13,8.412901e+07,99%


In [17]:
fig = px.line(
    policies[policies.phase_out_rate.isin([0.25, 0.5, 1])],
    "pov_ratio_guarantee",
    "cost_b",
    hover_data={"cost_b": ":.2f"},
    color="phase_out_rate_pct",
    color_discrete_map=COLOR_MAP,
    title="Cost of a means-tested benefit in New York City",
    labels=LABELS,
)
fig.add_hline(y=BUDGET / 1e9, line_dash="dash", line_color="gray")
fig.add_annotation(
    x=0.05,
    y=BUDGET / 1e9,
    align="left",
    text="Yang constraint<br>$1 billion cost",
    showarrow=False,
    font_color="gray",
)
fig.update_layout(
    yaxis_tickprefix="$",
    yaxis_ticksuffix="B",
    xaxis_tickformat="%",
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)
format_fig(fig)

In [18]:
fig = px.line(
    policies[policies.phase_out_rate.isin([0.25, 0.5, 1])],
    "pov_ratio_guarantee",
    "adult_reach",
    hover_data={"adult_reach": ":,.0f"},
    color="phase_out_rate_pct",
    color_discrete_map=COLOR_MAP,
    title="Adults receiving a means-tested benefit in New York City",
    labels=LABELS,
)
fig.add_hline(y=REACH, line_dash="dash", line_color="gray")
fig.add_annotation(
    x=0.07,
    y=REACH,
    align="left",
    text="Yang constraint<br>500,000 adult recipients",
    showarrow=False,
    font_color="gray",
)
fig.update_layout(
    yaxis_tickformat=",",
    xaxis_tickformat="%",
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)
format_fig(fig)

In [19]:
cost_curve = policies[policies.cost_b < 1].sort_values(
    "cost_b", ascending=False
    ).drop_duplicates("pov_ratio_guarantee").sort_values("pov_ratio_guarantee")
cost_curve["constraint"] = "$1 billion cost"
reach_curve = policies[policies.adult_reach < REACH].sort_values(
    "adult_reach", ascending=False
    ).drop_duplicates("pov_ratio_guarantee").sort_values("pov_ratio_guarantee")
reach_curve["constraint"] = "500,000 adult recipients"
cost_reach_curve = pd.concat([cost_curve, reach_curve])
fig = px.line(
    cost_reach_curve,
    "pov_ratio_guarantee",
    "phase_out_rate",
    hover_data={"cost_b": ":.2f", "adult_reach": ":,.0f"},
    color="constraint",
    color_discrete_map=COLOR_MAP,
    labels=LABELS,
    title="Guaranteed income policies that satisfy Yang's Basic Income for NYC constraints"
)
fig.update_layout(
    xaxis_tickformat="%",
    yaxis_tickformat="%",
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01)
    )
RATIO = 0.3
PS = 0.5
fig.add_annotation(x=RATIO, y=PS,
                   text="Maximum benefit: 30% of poverty threshold<br>Phase-out rate: 50%")
format_fig(fig)

In [20]:
opt = policies[(policies.pov_ratio_guarantee == RATIO) & (policies.phase_out_rate == PS)]
base = policy(0, 0)
comp = pd.DataFrame({"base": base, "reform": opt.iloc[0]}).reset_index().rename(columns={"index": "metric"})

def pct_diff(base, reform):
    return (reform - base) / base

comp["pct_diff"] = pct_diff(comp.base, comp.reform)
comp["yearly"] = comp.metric.apply(trend)
comp["years_progress"] = comp.pct_diff / comp.yearly
comp["name"] = comp.metric.map(LABELS)

,metric,base,reform,pct_diff,yearly,years_progress,name
0,adult_deep_poverty_rate,5.213446e-02,0.0396172,-0.240095,-0.050102,4.79211,Deep adult poverty rate
1,adult_poverty_rate,1.930477e-01,0.193048,-1.43776e-16,-0.031709,4.53424e-15,NaN
2,adult_reach,0.000000e+00,478342,inf,NaN,NaN,Adult recipients
3,child_deep_poverty_rate,4.039562e-02,0.0321026,-0.205295,-0.081702,2.51272,Deep child poverty rate
4,child_poverty_rate,2.351307e-01,0.235131,-1.18043e-16,-0.035742,3.30269e-15,NaN
5,child_reach,0.000000e+00,110863,inf,NaN,NaN,NaN
6,cost_b,0.000000e+00,0.99748,inf,NaN,NaN,Cost ($B)
7,deep_poverty_gap,1.445169e+09,5.97037e+08,-0.586874,-0.043708,13.4272,Deep poverty gap
8,deep_poverty_rate,4.970295e-02,0.0380607,-0.234237,-0.057959,4.0414,Deep poverty rate
9,gini,4.885562e-01,0.482643,-0.0121036,0.002775,-4.36166,NaN


In [29]:
PLOT_POV_MEASURES = [
    "poverty_rate",
    "deep_poverty_rate",
    "poverty_gap",
    "deep_poverty_gap",
]
fig = px.bar(
    comp[comp.metric.isin(PLOT_POV_MEASURES)].sort_values("pct_diff", ascending=False),
    "pct_diff",
    "name",
    orientation="h",
    text="pct_diff",
    title="Effect of Basic Income for NYC plan on poverty measures"
    )
fig.update_traces(marker_color=BLUE, texttemplate='%{text:.0%}')
fig.update_layout(xaxis_tickformat="%", xaxis_title="", yaxis_title="")
format_fig(fig)

In [31]:
fig = px.bar(
    comp[comp.metric.isin(PLOT_POV_MEASURES)].sort_values("years_progress"),
    "years_progress",
    "name",
    hover_data={"years_progress": ":.1f", "pct_diff": ":.1%", "yearly": ":.1%"},
    orientation="h",
    title="Effect of Basic Income for NYC on poverty measures in years of natural progress",
    labels=LABELS,
    )
fig.update_layout(
    xaxis_title="Policy impact / average yearly change from 2009-2019",
    yaxis_title="",
)
fig = format_fig(fig, show=False)
CONFIG = {"displayModeBar": False}
fig.write_html("impact_years.html", config=CONFIG)

In [20]:
# Determine individualized thresholds.
adult_thresh = s[(s.adult == 1) & (s.child == 0)].spmthresh.max()
child_thresh = ((s.spmthresh - s.adult * adult_thresh) / s.child).max()
print(adult_thresh, child_thresh)
s["spmthresh_ind"] = s.adult * adult_thresh + s.child * child_thresh
s[s.spmthresh == s.spmthresh_ind]

16470.0 8380.0


,year,spmfamunit,spmtotres,spmthresh,spmratio,spmwt,spmftotval,adult,child,max_transfer,transfer,spmtotres_r,spmthresh_ind
2775,2019,7689001,83471.0,24850.0,3.358994,1034.903333,111800,1,1,0.0,0.0,83471.0,24850.0
2795,2019,7711001,50790.0,16470.0,3.083789,1338.590000,73120,1,0,0.0,0.0,50790.0,16470.0
2972,2019,8035001,24600.0,16470.0,1.493625,928.990000,27653,1,0,0.0,0.0,24600.0,16470.0
3039,2019,8109001,70425.0,16470.0,4.275956,1957.620000,101965,1,0,0.0,0.0,70425.0,16470.0
3056,2019,8131001,75005.0,16470.0,4.554038,1073.023333,110011,1,0,0.0,0.0,75005.0,16470.0
3062,2019,8142001,59300.0,16470.0,3.600486,1470.310000,95210,1,0,0.0,0.0,59300.0,16470.0
3119,2019,8199001,16700.0,16470.0,1.013965,1649.740000,19476,1,0,0.0,0.0,16700.0,16470.0
3152,2019,8233001,143123.0,16470.0,8.689921,1085.913333,229201,1,0,0.0,0.0,143123.0,16470.0
3183,2019,8301005,21686.0,16470.0,1.316697,717.346667,31002,1,0,0.0,0.0,21686.0,16470.0
3231,2019,8404001,80899.0,16470.0,4.911900,751.550000,103627,1,0,0.0,0.0,80899.0,16470.0


In [21]:
SPMTHRESH_PREDICTORS = ["child", "adult"]
spmthresh_reg = sm.WLS(s.spmthresh, s[SPMTHRESH_PREDICTORS], s.spmwt).fit()
print(spmthresh_reg.params)
s["spmthresh_pred"] = spmthresh_reg.predict(s[SPMTHRESH_PREDICTORS])
s[SPMTHRESH_PREDICTORS + ["spmthresh", "spmthresh_ind", "spmthresh_pred"]]

child     5259.665502
adult    11075.127517
dtype: float64


,child,adult,spmthresh,spmthresh_ind,spmthresh_pred
0,0,2,21527.81574,32940.0,22150.255033
1,1,2,28996.05141,41320.0,27409.920536
2,1,1,23039.42354,24850.0,16334.793019
3,0,1,15267.95443,16470.0,11075.127517
4,1,2,28996.05141,41320.0,27409.920536
...,...,...,...,...,...
3918,4,2,43460.00000,66460.0,43188.917043
3919,2,2,35530.00000,49700.0,32669.586038
3920,1,3,32780.00000,57790.0,38485.048053
3921,2,2,35530.00000,49700.0,32669.586038


In [22]:
# Add cumulative adult population to identify bottom 500,000 adults.
pna = pn[pn.age >= 18].copy()
pna.sort_values("spmftotval", inplace=True)
pna["pop_cum"] = pna.asecwt.cumsum()

spmtotres_thresh = pna[pna.pop_cum < 500000].spmtotres.max()

pna[pna.pop_cum < 500000]

NameError: name 'pn' is not defined

In [7]:
sn = pn[["spmfamunit", "spmwt", "spmftotval", "spmtotres", "spmthresh"]].drop_duplicates().sort_values("spmftotval")
sn["spmwt_cum"] = sn.spmwt.cumsum()
spmftotval_thresh = sn[sn.spmwt_cum < 500000].spmftotval.max()
sn[sn.spmftotval <= spmftotval_thresh].spmwt.sum()
sn["ubi"] = np.where(sn.spmftotval <= spmftotval_thresh, 2000, 0)
sn["spmtotres_ubi"] = sn.spmtotres + sn.ubi

In [8]:
mdf.weighted_sum(sn, "ubi", "spmwt")

1004590420.0

In [9]:
pn = pn.merge(sn[["spmfamunit", "spmtotres_ubi"]], on="spmfamunit")


In [10]:
mdf.poverty_gap(sn, "spmtotres_ubi", "spmthresh", "spmwt") / 1e9

5.53524380132

In [11]:
mdf.poverty_gap(sn, "spmtotres", "spmthresh", "spmwt") / 1e9

6.256897865129999

In [12]:
mdf.deep_poverty_rate(pn, "spmtotres", "spmthresh", "asecwt")

0.04859213974110767

In [13]:
mdf.deep_poverty_rate(pn, "spmtotres_ubi", "spmthresh", "asecwt")

0.04356069904096139

In [14]:
mdf.poverty_rate(pn, "spmtotres_ubi", "spmthresh", "asecwt")

0.18354143352276175

In [15]:
mdf.poverty_rate(pn, "spmtotres", "spmthresh", "asecwt")

0.19196885996563795

In [16]:
def pct_chg(base, reform):
    return (reform - base) / base

pct_chg(mdf.poverty_rate(pn, "spmtotres", "spmthresh", "asecwt"),
mdf.poverty_rate(pn, "spmtotres_ubi", "spmthresh", "asecwt"))

-0.04389996609025385

In [17]:
pct_chg(mdf.deep_poverty_rate(pn, "spmtotres", "spmthresh", "asecwt"),
mdf.deep_poverty_rate(pn, "spmtotres_ubi", "spmthresh", "asecwt"))

-0.10354433303314309

In [18]:
pct_chg(mdf.gini(sn, "spmtotres", "spmwt"),
mdf.gini(sn, "spmtotres_ubi", "spmwt"))

-0.01087395327199434

In [20]:
p[(p.statefip == NYS_FIPS) & p.county.isin(NYC_FIPS)].asecwt.sum() / 1e6

NameError: name 'NYC_FIPS' is not defined

In [22]:
p[p.county.isin([NYS_FIPS * 1000 + i for i in NYC_FIPS])].groupby("county").asecwt.sum()

county
36005    1586280.10
36047    2634859.16
36061    1634742.52
36081    1869407.84
36085     497162.45
Name: asecwt, dtype: float64

In [20]:
p[p.county == 36061].asecwt.sum()

1634742.52